In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import glob
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf
from sklearn import metrics
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_curve

In [ ]:
def plot_roc(pred,y):
    fpr, tpr, thresholds = roc_curve(y, pred)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
classes = ['Tesla Model S Sedan 2012','Toyota Sequoia SUV 2012', 'Infiniti G Coupe IPL 2012', 'Lamborghini Aventador Coupe 2012', 'HUMMER H3T Crew Cab 2010']
print(len(classes))

# Without Transfer Learning (64 x 64)

In [ ]:
images = []
image_classes = []

for i in range(len(classes)):
    imagePaths = glob.glob('./dataset/custom/' + classes[i] + '/*.jpg')
    imageCount = 0
    for imgPath in imagePaths:
        if imageCount < 1000:
            imageCount += 1
            img = Image.open(imgPath).convert("RGB")
            img = img.resize((64, 64))
            img.load()
            data = np.asarray(img, dtype="int32")
            images.append(data)
            image_classes.append(i)
            

        
images = np.array(images)
image_classes = np.array(image_classes)

print(images.shape)
print(image_classes.shape)
print(len(classes))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images, image_classes, stratify=image_classes, random_state=123, shuffle=True, test_size=0.7)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


print(np.unique(image_classes))
print(np.unique(y_train))
print(np.unique(y_test))

In [ ]:
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)

print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)

plt.imshow(x_train[0])
print(classes[y_train[0]])

In [ ]:
# previous shape
print(y_train.shape)
print(y_test.shape)


In [ ]:
print(len(classes))
y_train = tf.keras.utils.to_categorical(y_train, len(classes))
y_test = tf.keras.utils.to_categorical(y_test, len(classes))


print(np.unique(y_train))
print(np.unique(y_test))

In [ ]:
# after shape
print(y_train.shape)
print(y_test.shape)

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255



print(x_train)
print(x_test)

In [ ]:
from tensorflow.keras.optimizers import SGD
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(x_train.shape[1], x_train.shape[2], 3)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(5, activation='softmax'))

In [ ]:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.0001))

model.fit(x_train, y_train, epochs=1, verbose=1 ,callbacks=[monitor], validation_data=(x_test, y_test))

In [ ]:
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)

y_true = np.argmax(y_test ,axis=1)

score = metrics.accuracy_score(y_true, pred)
print('Precision: {}'.format(score))

r1 = metrics.recall_score(y_true, pred, average='weighted')
print('Recall : {}'.format(r1))

f1 = metrics.f1_score(y_true, pred, average='weighted')
print('Averaged F1: {}'.format(f1))

print(metrics.classification_report(y_true, pred))

In [ ]:
fig = plt.figure(figsize=(20, 20))

rows = 5

images = x_test[0:rows];

for i in range(len(images)):
  fig.add_subplot(rows, 1, i+1)

  plt.imshow(x_test[i])
  plt.axis('off')
  plt.title("True Labels: {} \n Predicted Labels: {}".format(classes[y_true[i]], classes[pred[i]]))

# With Transfer Learning (64 x 64)

In [ ]:
images = []
image_classes = []

for i in range(len(classes)):
    imagePaths = glob.glob('./dataset/custom/' + classes[i] + '/*.jpg')
    imageCount = 0
    for imgPath in imagePaths:
        if imageCount < 1000:
            imageCount += 1
            img = Image.open(imgPath).convert("RGB")
            img = img.resize((64, 64))
            img.load()
            data = np.asarray(img, dtype="int32")
            images.append(data)
            image_classes.append(i)
            

        
images = np.array(images)
image_classes = np.array(image_classes)

print(images.shape)
print(image_classes.shape)
print(len(classes))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images, image_classes, stratify=image_classes, random_state=123, shuffle=True, test_size=0.7)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


print(np.unique(image_classes))
print(np.unique(y_train))
print(np.unique(y_test))

In [ ]:
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)

print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)

In [ ]:
plt.imshow(x_train[0])
print(classes[y_train[0]])

In [ ]:
# previous shape
print(y_train.shape)
print(y_test.shape)


In [ ]:
print(len(classes))
y_train = tf.keras.utils.to_categorical(y_train, len(classes))
y_test = tf.keras.utils.to_categorical(y_test, len(classes))


print(np.unique(y_train))
print(np.unique(y_test))

In [ ]:
# after shape
print(y_train.shape)
print(y_test.shape)

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255



print(x_train)
print(x_test)

In [ ]:
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

model = Sequential()

for layer in vgg_model.layers:
  model.add(layer)

model.summary()

In [ ]:
for layer in model.layers:
  layer.trainable = False

In [ ]:

model.add(Flatten())
model.add(Dense(1024))
model.add(Dense(1024))
model.add(Dense(len(classes), activation="softmax"))

###### model.summary()

In [ ]:
print(x_test.shape)
print(y_test.shape)

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.0001))

model.fit(x_train, y_train, epochs=1, verbose=1 ,callbacks=[monitor], validation_data=(x_test, y_test))

In [ ]:
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)

y_true = np.argmax(y_test ,axis=1)

score = metrics.accuracy_score(y_true, pred)
print('Precision: {}'.format(score))

r1 = metrics.recall_score(y_true, pred, average='weighted')
print('Recall : {}'.format(r1))

f1 = metrics.f1_score(y_true, pred, average='weighted')
print('Averaged F1: {}'.format(f1))

print(metrics.classification_report(y_true, pred))

In [ ]:
fig = plt.figure(figsize=(20, 20))

rows = 5

images = x_test[0:rows];

for i in range(len(images)):
  fig.add_subplot(rows, 1, i+1)

  plt.imshow(x_test[i])
  plt.axis('off')
  plt.title("True Labels: {} \n Predicted Labels: {}".format(classes[y_true[i]], classes[pred[i]]))

# Without Transfer Learning (200 x 200)


In [ ]:
images = []
image_classes = []

for i in range(len(classes)):
    imagePaths = glob.glob('./dataset/custom/' + classes[i] + '/*.jpg')
    imageCount = 0
    for imgPath in imagePaths:
        if imageCount < 1000:
            imageCount += 1
            img = Image.open(imgPath).convert("RGB")
            img = img.resize((64, 64))
            img.load()
            data = np.asarray(img, dtype="int32")
            images.append(data)
            image_classes.append(i)
            

        
images = np.array(images)
image_classes = np.array(image_classes)

print(images.shape)
print(image_classes.shape)
print(len(classes))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images, image_classes, stratify=image_classes, random_state=123, shuffle=True, test_size=0.7)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


print(np.unique(image_classes))
print(np.unique(y_train))
print(np.unique(y_test))

In [ ]:
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)

print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)

plt.imshow(x_train[0])
print(classes[y_train[0]])

In [ ]:
# previous shape
print(y_train.shape)
print(y_test.shape)


In [ ]:
print(len(classes))
y_train = tf.keras.utils.to_categorical(y_train, len(classes))
y_test = tf.keras.utils.to_categorical(y_test, len(classes))


print(np.unique(y_train))
print(np.unique(y_test))

In [ ]:
# after shape
print(y_train.shape)
print(y_test.shape)

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255



print(x_train)
print(x_test)

In [ ]:
from tensorflow.keras.optimizers import SGD
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(x_train.shape[1], x_train.shape[2], 3)))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(5, activation='softmax'))

In [ ]:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.0001))

model.fit(x_train, y_train, epochs=1, verbose=1 ,callbacks=[monitor], validation_data=(x_test, y_test))

In [ ]:
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)

y_true = np.argmax(y_test ,axis=1)

score = metrics.accuracy_score(y_true, pred)
print('Precision: {}'.format(score))

r1 = metrics.recall_score(y_true, pred, average='weighted')
print('Recall : {}'.format(r1))

f1 = metrics.f1_score(y_true, pred, average='weighted')
print('Averaged F1: {}'.format(f1))

print(metrics.classification_report(y_true, pred))

In [ ]:
fig = plt.figure(figsize=(20, 20))

rows = 5

images = x_test[0:rows];

for i in range(len(images)):
  fig.add_subplot(rows, 1, i+1)

  plt.imshow(x_test[i])
  plt.axis('off')
  plt.title("True Labels: {} \n Predicted Labels: {}".format(classes[y_true[i]], classes[pred[i]]))

# With Transfer Learning (200 x 200)

In [ ]:
images = []
image_classes = []

for i in range(len(classes)):
    imagePaths = glob.glob('./dataset/custom/' + classes[i] + '/*.jpg')
    imageCount = 0
    for imgPath in imagePaths:
        if imageCount < 1000:
            imageCount += 1
            img = Image.open(imgPath).convert("RGB")
            img = img.resize((200, 200))
            img.load()
            data = np.asarray(img, dtype="int32")
            images.append(data)
            image_classes.append(i)
            

        
images = np.array(images)
image_classes = np.array(image_classes)

print(images.shape)
print(image_classes.shape)
print(len(classes))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images, image_classes, stratify=image_classes, random_state=123, shuffle=True, test_size=0.7)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


print(np.unique(image_classes))
print(np.unique(y_train))
print(np.unique(y_test))

In [ ]:
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)

print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)

In [ ]:
plt.imshow(x_train[0])
print(classes[y_train[0]])

In [ ]:
# previous shape
print(y_train.shape)
print(y_test.shape)


In [ ]:
print(len(classes))
y_train = tf.keras.utils.to_categorical(y_train, len(classes))
y_test = tf.keras.utils.to_categorical(y_test, len(classes))


print(np.unique(y_train))
print(np.unique(y_test))

In [ ]:
# after shape
print(y_train.shape)
print(y_test.shape)

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255



print(x_train)
print(x_test)

In [ ]:
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

model = Sequential()

for layer in vgg_model.layers:
  model.add(layer)

model.summary()

In [ ]:
for layer in model.layers:
  layer.trainable = False

In [ ]:

model.add(Flatten())
model.add(Dense(1024))
model.add(Dense(1024))
model.add(Dense(len(classes), activation="softmax"))

In [ ]:
print(x_test.shape)
print(y_test.shape)

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')

model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.0001))

model.fit(x_train, y_train, epochs=1, verbose=1 ,callbacks=[monitor], validation_data=(x_test, y_test))

In [ ]:
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)

y_true = np.argmax(y_test ,axis=1)

score = metrics.accuracy_score(y_true, pred)
print('Precision: {}'.format(score))

r1 = metrics.recall_score(y_true, pred, average='weighted')
print('Recall : {}'.format(r1))

f1 = metrics.f1_score(y_true, pred, average='weighted')
print('Averaged F1: {}'.format(f1))

print(metrics.classification_report(y_true, pred))

In [ ]:
fig = plt.figure(figsize=(20, 20))

rows = 5

images = x_test[0:rows];

for i in range(len(images)):
  fig.add_subplot(rows, 1, i+1)

  plt.imshow(x_test[i])
  plt.axis('off')
  plt.title("True Labels: {} \n Predicted Labels: {}".format(classes[y_true[i]], classes[pred[i]]))